## インタラクティブ Phi 3 Mini 4K インストラクトチャットボットとWhisper

### はじめに:
インタラクティブ Phi 3 Mini 4K インストラクトチャットボットは、Microsoft Phi 3 Mini 4K インストラクトデモとテキストまたは音声入力を使ってやり取りできるツールです。このチャットボットは、翻訳、天気情報の更新、一般的な情報収集など、さまざまなタスクに利用できます。


In [ ]:
#Install required Python Packages
!pip install accelerate
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install flash-attn --no-build-isolation', env={'FLASH_ATTENTION_SKIP_CUDA_BUILD': "TRUE"}, shell=True
!pip install transformers
!pip install wheel
!pip install gradio
!pip install pydub==0.25.1
!pip install edge-tts
!pip install openai-whisper==20231117
!pip install ffmpeg==1.4
# from IPython.display import clear_output
# clear_output()

In [ ]:
# Checking to see if Cuda support is available 
# Output True = Cuda
# Output False = No Cuda (installing Cuda will be required to run the model on GPU)
import os 
import torch
print(torch.cuda.is_available())


[Huggingfaceアクセストークンを作成する](https://huggingface.co/settings/tokens)

新しいトークンを作成  
新しい名前を入力  
書き込み権限を選択  
トークンをコピーして安全な場所に保存


以下はPythonコードで、主に2つのタスクを実行します: `os`モジュールのインポートと環境変数の設定。

1. `os`モジュールのインポート:
   - Pythonの`os`モジュールは、オペレーティングシステムとやり取りするための方法を提供します。環境変数へのアクセス、ファイルやディレクトリの操作など、さまざまなOS関連のタスクを実行することができます。
   - このコードでは、`import`文を使用して`os`モジュールをインポートしています。この文により、現在のPythonスクリプト内で`os`モジュールの機能が利用可能になります。

2. 環境変数の設定:
   - 環境変数は、オペレーティングシステム上で動作するプログラムがアクセスできる値です。これは、複数のプログラムで使用される設定情報やその他のデータを保存する方法です。
   - このコードでは、`os.environ`辞書を使用して新しい環境変数を設定しています。辞書のキーは`'HF_TOKEN'`で、値は`HUGGINGFACE_TOKEN`変数から割り当てられます。
   - `HUGGINGFACE_TOKEN`変数は、このコードスニペットの直前で定義されており、`#@param`構文を使用して文字列値`"hf_**************"`が割り当てられています。この構文は、Jupyterノートブックでユーザー入力やパラメータ設定を直接ノートブックインターフェースで行うためによく使用されます。
   - `'HF_TOKEN'`環境変数を設定することで、プログラムの他の部分や同じオペレーティングシステム上で動作する他のプログラムからアクセス可能になります。

全体として、このコードは`os`モジュールをインポートし、`HUGGINGFACE_TOKEN`変数で提供された値を使用して`'HF_TOKEN'`という名前の環境変数を設定しています。


In [ ]:
import os
# set the Hugging Face Token from 
# add the Hugging Face Token to the environment variables
HUGGINGFACE_TOKEN = "Enter Hugging Face Key" #@param {type:"string"}
os.environ['HF_TOKEN']HUGGINGFACE_TOKEN

このコードスニペットは、Jupyter NotebookやIPythonで現在のセルの出力をクリアするための関数、clear_outputを定義しています。コードを分解してその機能を理解してみましょう。

関数clear_outputは、waitという名前の1つのパラメータを取ります。このパラメータはブール値で、デフォルトではFalseに設定されています。waitは、新しい出力が既存の出力を置き換えるまで待つかどうかを決定します。

この関数自体は、現在のセルの出力をクリアするために使用されます。Jupyter NotebookやIPythonでは、セルが出力を生成すると（例えば、テキストの印刷やグラフィカルなプロットなど）、その出力はセルの下に表示されます。clear_output関数を使用すると、その出力をクリアすることができます。

コードスニペットでは、関数の実装は省略記号（...）で示されており、実際のコードは提供されていません。この省略記号は、出力をクリアする処理を行う実際のコードのプレースホルダーを表しています。関数の実装は、Jupyter NotebookやIPythonのAPIと連携してセルの既存の出力を削除することを含む可能性があります。

全体として、この関数はJupyter NotebookやIPythonで現在のセルの出力をクリアする便利な方法を提供します。これにより、インタラクティブなコーディングセッション中に表示される出力を管理および更新しやすくなります。


In [ ]:
# Download Phi-3-mini-4k-instruct model & Whisper Tiny
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

#whisper for speech to text()
import whisper
select_model ="tiny" # ['tiny', 'base']
whisper_model = whisper.load_model(select_model)

#from IPython.display import clear_output
#clear_output()

Edge TTSサービスを使用してテキスト読み上げ（TTS）を実行します。以下の関数実装を順番に説明します：

1. `calculate_rate_string(input_value)`：この関数は入力値を受け取り、TTS音声の速度文字列を計算します。入力値は読み上げ速度を表し、1が通常速度を意味します。この関数は、入力値から1を引き、100を掛けて速度を計算し、入力値が1以上かどうかで符号を決定します。関数は速度文字列を"{sign}{rate}"の形式で返します。

2. `make_chunks(input_text, language)`：この関数は入力テキストと言語を受け取り、言語固有のルールに基づいてテキストをチャンクに分割します。この実装では、言語が「英語」の場合、テキストをピリオド（"."）で分割し、前後の空白を削除します。その後、各チャンクにピリオドを追加し、フィルタリングされたチャンクのリストを返します。

3. `tts_file_name(text)`：この関数は入力テキストに基づいてTTS音声ファイルの名前を生成します。テキストの末尾のピリオドを削除し（存在する場合）、小文字に変換し、前後の空白を削除し、スペースをアンダースコアに置き換えます。その後、テキストが25文字を超える場合は切り詰め、空の場合はそのまま使用します。最後に、[`uuid`]モジュールを使用してランダムな文字列を生成し、切り詰めたテキストと組み合わせて"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"形式のファイル名を作成します。

4. `merge_audio_files(audio_paths, output_path)`：この関数は複数の音声ファイルを1つの音声ファイルに結合します。音声ファイルのパスのリストと出力パスを受け取ります。関数は空の[`AudioSegment`]オブジェクトを初期化し、各音声ファイルのパスを反復処理して、`pydub`ライブラリの`AudioSegment.from_file()`メソッドを使用して音声ファイルを読み込み、[`merged_audio`]オブジェクトに追加します。最後に、指定された出力パスにMP3形式で結合された音声をエクスポートします。

5. `edge_free_tts(chunks_list, speed, voice_name, save_path)`：この関数はEdge TTSサービスを使用してTTS操作を実行します。テキストチャンクのリスト、読み上げ速度、音声名、保存パスを受け取ります。チャンク数が1より多い場合、個々のチャンク音声ファイルを保存するディレクトリを作成します。その後、各チャンクを反復処理し、`calculate_rate_string()`関数、音声名、チャンクテキストを使用してEdge TTSコマンドを構築し、`os.system()`関数を使用してコマンドを実行します。コマンドが正常に実行されると、生成された音声ファイルのパスをリストに追加します。すべてのチャンクを処理した後、`merge_audio_files()`関数を使用して個々の音声ファイルを結合し、指定された保存パスに結合された音声を保存します。チャンクが1つだけの場合は、直接Edge TTSコマンドを生成して音声を保存パスに保存します。最後に、生成された音声ファイルの保存パスを返します。

6. `random_audio_name_generate()`：この関数は[`uuid`]モジュールを使用してランダムな音声ファイル名を生成します。ランダムなUUIDを生成し、文字列に変換して最初の8文字を取得し、".mp3"拡張子を追加してランダムな音声ファイル名を返します。

7. `talk(input_text)`：この関数はTTS操作を実行するメインエントリポイントです。入力テキストを受け取ります。まず、入力テキストの長さを確認し、長文（600文字以上）かどうかを判断します。長さと`translate_text_flag`変数の値に基づいて言語を決定し、`make_chunks()`関数を使用してテキストチャンクのリストを生成します。その後、`random_audio_name_generate()`関数を使用して音声ファイルの保存パスを生成します。最後に、`edge_free_tts()`関数を呼び出してTTS操作を実行し、生成された音声ファイルの保存パスを返します。

これらの関数は連携して、入力テキストをチャンクに分割し、音声ファイルの名前を生成し、Edge TTSサービスを使用してTTS操作を実行し、個々の音声ファイルを1つの音声ファイルに結合します。


In [ ]:
#@title Edge TTS
def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"


def make_chunks(input_text, language):
    language="English"
    if language == "English":
      temp_list = input_text.strip().split(".")
      filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
      if temp_list[-1].strip():
          filtered_list.append(temp_list[-1].strip())
      return filtered_list


import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # Initialize an empty AudioSegment
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through each audio file path
    for audio_path in audio_paths:
        # Load the audio file using Pydub
        audio = AudioSegment.from_file(audio_path)

        # Append the current audio file to the merged_audio
        merged_audio += audio

    # Export the merged audio to the specified output path
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  # print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      print(i)
      edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'
      print(edge_command)
      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    # print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path

# text = "This is Microsoft Phi 3 mini 4k instruct Demo" Simply update the text variable with the text you want to convert to speech
text = 'This is Microsoft Phi 3 mini 4k instruct Demo'  # @param {type: "string"}
Language = "English" # @param ['English']
# Gender of voice simply change from male to female and choose the voice you want to use
Gender = "Female"# @param ['Male', 'Female']
female_voice="en-US-AriaNeural"# @param["en-US-AriaNeural",'zh-CN-XiaoxiaoNeural','zh-CN-XiaoyiNeural']
speed = 1  # @param {type: "number"}
translate_text_flag  = False
if len(text)>=600:
  long_sentence = True
else:
  long_sentence = False

# long_sentence = False # @param {type:"boolean"}
save_path = ''  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    voice_name="en-US-ChristopherNeural"
  if Gender=="Female":
    voice_name=female_voice
    # voice_name="en-US-AriaNeural"


if translate_text_flag:
  input_text=text
  # input_text=translate_text(text, Language)
  # print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
# edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
# from IPython.display import clear_output
# clear_output()
# from IPython.display import Audio
# Audio(edge_save_path, autoplay=True)

from IPython.display import clear_output
from IPython.display import Audio
if not os.path.exists("/content/audio"):
    os.mkdir("/content/audio")
import uuid
def random_audio_name_generate():
  random_uuid = uuid.uuid4()
  audio_extension = ".mp3"
  random_audio_name = str(random_uuid)[:8] + audio_extension
  return random_audio_name
def talk(input_text):
  global translate_text_flag,Language,speed,voice_name
  if len(input_text)>=600:
    long_sentence = True
  else:
    long_sentence = False

  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  save_path="/content/audio/"+random_audio_name_generate()
  edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
  return edge_save_path


edge_save_path=talk(text)
Audio(edge_save_path, autoplay=True)

2つの関数、convert_to_textとrun_text_promptの実装、および2つのクラス、strとAudioの宣言について。

convert_to_text関数はaudio_pathを入力として受け取り、whisper_modelというモデルを使用して音声をテキストに変換します。この関数はまずgpuフラグがTrueかどうかを確認します。Trueの場合、whisper_modelはword_timestamps=True、fp16=True、language='English'、task='translate'といった特定のパラメータで使用されます。gpuフラグがFalseの場合、whisper_modelはfp16=Falseで使用されます。変換されたテキストは'scan.txt'という名前のファイルに保存され、テキストとして返されます。

run_text_prompt関数はmessageとchat_historyを入力として受け取ります。この関数はphi_demo関数を使用して、入力されたメッセージに基づいてチャットボットからの応答を生成します。生成された応答はtalk関数に渡され、応答を音声ファイルに変換してファイルパスを返します。Audioクラスは音声ファイルを表示し再生するために使用されます。音声はIPython.displayモジュールのdisplay関数を使用して表示され、Audioオブジェクトはautoplay=Trueパラメータで作成されるため、音声は自動的に再生されます。chat_historyは入力メッセージと生成された応答で更新され、空の文字列と更新されたchat_historyが返されます。

strクラスはPythonの組み込みクラスで、文字列のシーケンスを表します。このクラスは、文字列を操作したり処理するためのさまざまなメソッドを提供します。例えば、capitalize、casefold、center、count、encode、endswith、expandtabs、find、format、index、isalnum、isalpha、isascii、isdecimal、isdigit、isidentifier、islower、isnumeric、isprintable、isspace、istitle、isupper、join、ljust、lower、lstrip、partition、replace、removeprefix、removesuffix、rfind、rindex、rjust、rpartition、rsplit、rstrip、split、splitlines、startswith、strip、swapcase、title、translate、upper、zfillなどがあります。これらのメソッドを使用すると、検索、置換、フォーマット、文字列の操作などが可能です。

Audioクラスはカスタムクラスで、音声オブジェクトを表します。このクラスはJupyter Notebook環境で音声プレーヤーを作成するために使用されます。クラスはdata、filename、url、embed、rate、autoplay、normalizeといったさまざまなパラメータを受け取ります。dataパラメータはnumpy配列、サンプルのリスト、ファイル名やURLを表す文字列、または生のPCMデータを指定できます。filenameパラメータは音声データを読み込むローカルファイルを指定し、urlパラメータは音声データをダウンロードするURLを指定します。embedパラメータは音声データをデータURIとして埋め込むか、元のソースから参照するかを決定します。rateパラメータは音声データのサンプリングレートを指定します。autoplayパラメータは音声を自動的に再生するかどうかを決定します。normalizeパラメータは音声データを最大可能範囲に正規化するかどうかを指定します。Audioクラスはreloadメソッドを提供してファイルやURLから音声データを再読み込みすることができ、src_attr、autoplay_attr、element_id_attrといった属性を使用してHTML内の音声要素の対応する属性を取得できます。

これらの関数とクラスは、音声をテキストに変換したり、チャットボットからの音声応答を生成したり、Jupyter Notebook環境で音声を表示・再生するために使用されます。


In [ ]:
#@title Run gradio app
def convert_to_text(audio_path):
  gpu=True
  if gpu:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=True,language='English',task='translate')
  else:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=False,language='English',task='translate')
  with open('scan.txt', 'w') as file:
    file.write(str(result))
  return result["text"]


import gradio as gr
from IPython.display import Audio, display
def run_text_prompt(message, chat_history):
    bot_message = phi_demo(message)
    edge_save_path=talk(bot_message)
    # print(edge_save_path)
    display(Audio(edge_save_path, autoplay=True))

    chat_history.append((message, bot_message))
    return "", chat_history


def run_audio_prompt(audio, chat_history):
    if audio is None:
        return None, chat_history
    print(audio)
    message_transcription = convert_to_text(audio)
    _, chat_history = run_text_prompt(message_transcription, chat_history)
    return None, chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat with Phi 3 mini 4k instruct")

    msg = gr.Textbox(label="Ask anything")
    msg.submit(run_text_prompt, [msg, chatbot], [msg, chatbot])

    with gr.Row():
        audio = gr.Audio(sources="microphone", type="filepath")

        send_audio_button = gr.Button("Send Audio", interactive=True)
        send_audio_button.click(run_audio_prompt, [audio, chatbot], [audio, chatbot])

demo.launch(share=True,debug=True)


---

**免責事項**:  
この文書は、AI翻訳サービス [Co-op Translator](https://github.com/Azure/co-op-translator) を使用して翻訳されています。正確性を追求しておりますが、自動翻訳には誤りや不正確な部分が含まれる可能性があります。元の言語で記載された文書が正式な情報源とみなされるべきです。重要な情報については、専門の人間による翻訳を推奨します。この翻訳の使用に起因する誤解や誤解釈について、当方は一切の責任を負いません。
